In [6]:
import sys
sys.path.append("C://dev/ISV_data_gathering/")
sys.path.append("C://dev/")

from translation_aux import prepare_slovnik
from isv_translate import get_slovnik, download_slovnik

In [7]:
from isv_nlp_utils.slovnik import get_slovnik

In [11]:
from isv_nlp_utils import constants

In [8]:
dfs = get_slovnik()
slovnik = dfs['words']
prepare_slovnik(slovnik)


Found 'slovnik.pkl' file, using it


In [10]:
from razdel import tokenize



In [12]:
etm_morph = constants.create_analyzers_for_every_alphabet(r"C:\dev\ISV_data_gathering\\")['etm']


In [13]:
def extract_stem_prefix_suffix(word, word_parse, isv_dict):
    
    paradigm_num = word_parse[4][0][2]
    form_num = word_parse[4][0][3]
    paradigm = isv_dict.build_paradigm_info(paradigm_num)
    stem = isv_dict.build_stem(isv_dict.paradigms[paradigm_num], form_num, word)

    pref = paradigm[form_num][0]
    suff = paradigm[form_num][2]
    
    return pref, stem, suff

isv_dict = etm_morph._units[0][0].dict

word = "putniku"
word_parse = etm_morph.parse(word)[0]

extract_stem_prefix_suffix(word, word_parse, isv_dict)

('', 'putni', 'ku')

In [14]:
from collections import Counter

def extract_stem_prefix_suffix(word, word_parse, isv_dict, postprocess_freq_thres=1.0):
    
    paradigm_num = word_parse[4][0][2]
    form_num = word_parse[4][0][3]
    paradigm = isv_dict.build_paradigm_info(paradigm_num)
    stem = isv_dict.build_stem(isv_dict.paradigms[paradigm_num], form_num, word)

    pref = paradigm[form_num][0]
    suff = paradigm[form_num][2]

    fixed_stem = stem
    fixed_suf = suff

    for pref_len in range(1, 5):
        freq = Counter([suf[:pref_len] for (pref, tags, suf) in paradigm])
        most_common_pref, its_freq = freq.most_common(1)[0]
        if its_freq / len(paradigm) >= postprocess_freq_thres:
            fixed_stem = stem + most_common_pref
            fixed_suf = suff[pref_len:]
            # print(stem, suff, "->", fixed_stem, fixed_suf)
    
    return pref, fixed_stem, fixed_suf


In [70]:
COLS = ['id', 'isv', 'addition', 'en', 'ru', 'bg', 'hr', 'sr', 'frequency',
       'pos']

In [71]:

slovnik[slovnik.partOfSpeech.str.contains("adj")][COLS]

,id,isv,addition,en,ru,bg,hr,sr,frequency,pos
9,36565,abhazsky,NaN,"Abkhaz, Abkhazian",абхазский,абхазки,abhaski,абхазијски,0.444167,adj
26,6120,absolutny,NaN,"absolute, utter, sheer","абсолютный, полнейший","абсолютен, чист, пълен, истински","apsolutan, potpun, neograničen",апсолутан,1.17125,adj
32,19627,abstraktny,NaN,abstract,"абстрактный, отвлеченный",абстрактен,apstraktan,апстрактан,1.188333,adj
35,6125,absurdny,NaN,absurd,"абсурдный, нелепый, несуразный",абсурден,apsurdan,"апсурдан, бесмислен",1.170833,adj
44,24071,adekvatny,NaN,adequate,"адекватный, удовлетворяющий",адекватен,"odgovarajući, prikladan, dovoljan, adekvatan","одговарајући, прикладан, довољан, адекватан",1.493333,adj
...,...,...,...,...,...,...,...,...,...,...
17934,-37025,gazovy,NaN,gas,газовый,!gasni,!gasni,"гасни, који користи гас",!,adj
17936,-37028,sněžny,NaN,snowy,"снежный, заснеженный",!снежно,"!snježna, snježna, snježna","снежни, снијежни, сњежни",!,adj
17939,-37031,drožđevy,NaN,yeast,дрожжевой,!дрожди,!kvasci,"квашчев, који садржи квасац, од квасца",!,adj
17945,-37037,kvasny,NaN,yeast,дрожжевой,!дрожди,!kvasci,"квашчев, који садржи квасац, од квасца",!,adj


In [50]:
from isv_nlp_utils.constants import ALPHABET_LETTERS

ALPHABET_LETTERS

{'lat': 'abcdefghijklmnoprstuvyzěčšž',
 'cyr': 'абвгдежзиклмнопрстуфхцчшыєјљњ',
 'etm': 'abcdefghijklmnoprstuvyzåćčďėęěľńŕśšťųźžȯʒ'}

In [138]:
VOWELS = "aeiouųåėęěȯy"
CONSONANTS = "bcdfghjklmnprstvzćčďľńŕśšťźžʒđ"



In [139]:
from itertools import takewhile

def get_consonants_ending(word):
    if word[-3:] in [" na", " od", " za"]:
        word = word[:-3]
    if word[-1] not in 'yi':
        return word
    word = word[:-1]
    cons = "".join(takewhile(lambda c: c in CONSONANTS, reversed(word)))
    return cons[::-1]

get_consonants_ending("potrěbny"), get_consonants_ending("bezzzųby")

('bn', 'b')

In [140]:
adjs = slovnik[slovnik.partOfSpeech.str.contains("adj")][COLS].copy()

adjs["last"] = adjs.isv.apply(get_consonants_ending)

In [141]:
adjs["last"]

9        zsk
26        tn
32       ktn
35       rdn
44        tn
        ... 
17934      v
17936     žn
17939      v
17945     sn
17966      g
Name: last, Length: 2886, dtype: object

In [142]:
vc = adjs["last"].value_counts()
vc[vc == 1]

brn     1
lgk     1
tvŕd    1
rfn     1
lssk    1
       ..
ndn     1
rtn     1
lzk     1
jk      1
lt      1
Name: last, Length: 80, dtype: int64

In [143]:
adjs.query('last in @vc[@vc==1].index', engine='python')

,id,isv,addition,en,ru,bg,hr,sr,frequency,pos,last
537,24425,babin,NaN,grandma’s,бабушкин,бабин,bakin,"бабин, бакин",0.46,adj,babin
766,24646,bezmozgy,NaN,brainless,безмозглый,"безмозъчен, празноглав, тъп","praznoglav, šupljoglav, glup, ograničen","безмозган, глуп, блесав, празноглав",0.627917,adj,zg
976,15253,bobŕji,NaN,beaver,бобровый,бобров,dabarski,дабарски,0.624167,adj,bŕj
1086,6262,božji,NaN,God’s,божий,"божи, Божи",božji,божји,2.171667,adj,žj
1111,24801,bratov,NaN,brother’s,"брата, братов, братний",братов,"bratov, bratovljev","братов, братовљев",0.950833,adj,bratov
...,...,...,...,...,...,...,...,...,...,...,...
16966,78,vųgrsky,NaN,Hungarian,венгерский,"унгарски, маджарски",mađarski,мађарски,2.020417,adj,grsk
17026,20302,vysši,NaN,higher,"выше, более высокий",висш,viši,виши,2.340833,adj,sš
17679,31640,zlověšći,NaN,ominous,зловещий,"зловещ, злокобен","zlosutan, zlokoban, prijeteći","злосутан, злокобан",0.805833,adj,šć
17684,7508,zmiji,NaN,"snake, viperous",змеиный,змийски,zmijski,змијски,0.605,adj,j


In [144]:
adjs.query('last in @vc[@vc==2].index', engine='python')

,id,isv,addition,en,ru,bg,hr,sr,frequency,pos,last
32,19627,abstraktny,NaN,abstract,"абстрактный, отвлеченный",абстрактен,apstraktan,апстрактан,1.188333,adj,ktn
35,6125,absurdny,NaN,absurd,"абсурдный, нелепый, несуразный",абсурден,apsurdan,"апсурдан, бесмислен",1.170833,adj,rdn
765,20351,bezmȯlvny,NaN,speechless,безмолвный,"онемял, безмълвен","bezglasan, nijem, zanijemio","нем, онемео",0.407917,adj,lvn
997,6629,bogatši,NaN,richer,"богаче, более богатый",по-богат,bogatiji,богатији,1.639583,adj,tš
1232,24878,buŕny,NaN,"turbulent, stormy","бурный, штормовой, ураганный","бурлив, неспокоен",buran,буран,0.951667,adj,ŕn
1774,25184,delfińji,NaN,dolphin’s,дельфиний,делфински,"delfinski, dupinski",делфински,0.50875,adj,ńj
2430,4459,duži,NaN,great,"большой, крупный, великий","голям, велик","velik, veliki","велик, велики, голем",2.360417,adj,ž
2534,4757,egiptsky,NaN,Egyptian,египетский,египетски,egipatski,египатски,1.90375,adj,ptsk
2668,22777,etiopsky,NaN,Ethiopian,эфиопский,етиопски,etiopski,етиопски,0.840417,adj,psk
2688,1905,"evropejsky, evropsky",NaN,European,европейский,европейски,"europski, evropski",европски,2.44625,adj,psk


In [133]:
adjs.query('last in @vc[@vc==3].index', engine='python')

,id,isv,addition,en,ru,bg,hr,sr,frequency,pos,last
701,2864,bělorussky,NaN,Belarusian,белорусский,беларуски,bjeloruski,белоруски,1.305833,adj,ssk
722,3971,bezbarvny,NaN,"colourless, colorless",бесцветный,"блед, безцветен",bezbojan,безбојан,1.035,adj,rvn
2115,19677,doľnosŕbsky,NaN,Lower Sorbian,нижнелужицкий,долнолужишки,"donjolužički, donjolužičkosrpski","доњелужички, доњелужичкосрпски, доњесрпски",0.054167,adj,sŕbsk
2125,3573,dȯlžny,NaN,due,"обязательный, должный, подлежащий исполнению",задължителен,dužan,дужан,1.815833,adj,lžn
2126,6161,dȯlžny,NaN,"should, ought to, bound to","должен, обязанный","длъжен, задължен","obavezan, obvezatan","дужан, обавезан",1.63,adj,lžn
2183,15304,doråstly,NaN,adult,"взрослый, зрелый",възрастен,"odrastao, punoljetan, zreo, odrasla osoba",одрастао,1.924583,adj,stl
2463,15309,dvojostry,NaN,double-edged,обоюдоострый,"двуостър, с две остриета",dvosjekli,двосекли,0.770833,adj,str
2487,17363,dvusmysľny,NaN,ambiguous,"двусмысленный, неоднозначный",двусмислен,"dvosmislen, dvoznačan","двосмислен, двозначан",0.655417,adj,smysľn
3151,19709,gornosŕbsky,NaN,Upper Sorbian,верхнелужицкий,горнолужишки,gornjolužički,лужичкосрпски,0.139167,adj,sŕbsk
3490,22387,hondurassky,NaN,Honduran,гондурасский,хондураски,honduraški,хондурашки,0.288333,adj,ssk


In [93]:
for word in [
'aluminijevy',
'basnjesloveny', 
'bělobrådy', 
'bělopery',
'bělovlåsy', 
'běly', 
'beznogy', 
'bezpokojęći', 
'bogaty', 
'ćuđi',
'daleky', 
'dopustimy', 
'duži', 
'gluhy', 
'glupy', 
'gruby', 
'loši',
'zmiji'
]:
    print(word, " -> ", word[:-1], "(BG HR: )")

aluminijevy  ->  aluminijev (BG HR: )
basnjesloveny  ->  basnjesloven (BG HR: )
bělobrådy  ->  bělobråd (BG HR: )
bělopery  ->  běloper (BG HR: )
bělovlåsy  ->  bělovlås (BG HR: )
běly  ->  běl (BG HR: )
beznogy  ->  beznog (BG HR: )
bezpokojęći  ->  bezpokojęć (BG HR: )
bogaty  ->  bogat (BG HR: )
ćuđi  ->  ćuđ (BG HR: )
daleky  ->  dalek (BG HR: )
dopustimy  ->  dopustim (BG HR: )
duži  ->  duž (BG HR: )
gluhy  ->  gluh (BG HR: )
glupy  ->  glup (BG HR: )
gruby  ->  grub (BG HR: )
loši  ->  loš (BG HR: )
zmiji  ->  zmij (BG HR: )


In [ ]:
Iztvoreńje kråtkoj formy je prosto, kogda na koncu jest same jedna sųglaska. Potrěbno je odstraniti samoglasky na koncu i prěměniti drugų samoglåskų na ėȯ ako li jest možno:

basnjesloveny  ->  basnjeslovėn (HR: basnoslovan)
bělobrådy  ->  bělobråd (HR: bjelobrad)
bělopery  ->  běloper (HR: bjeloper, SR: белоперан)
bělovlåsy  ->  bělovlås (BG: беловлас)
běly  ->  běl (HR: bijel)
beznogy  ->  beznog (BG: безног, HR: beznožan)
bogaty  ->  bogat (HR: bogat)
ćuđi  ->  ćuđ (HR: tuđ, RU: чужд)
daleky  ->  dalek (HR: dalek)
dopustimy  ->  dopustim (BG: допустим)
duži  ->  duž (RU: дюж)
gluhy  ->  gluh (HR: gluh)
glupy  ->  glup (BG HR: glup)
gruby  ->  grub (BG HR: grub)
loši  ->  loš (BG HR: loš)

vidžu že južne jezyky čęsto imajut dodatok "-an/-en", ale on ne jest obvězėn.

Imam problem, kogda sųglåska na koncu jest V, Ć, J:

V:
aluminijevy  ->  aluminijev (BG: алуминиев) (ili aluminijėv?) 
takože višnjevy ->  HR: višnjev, BG: вишнен
I ja absolutno ne věm, čto dělati s napriklad "basnjevy beževy bojevy sětevy"

Ć:
vseznajųći -> BG: всезнаещ, HR: sveznajuć
ale
bųdųći -> BG: бъдещ HR: budući
imųći -> HR: imućan
I ne věm odnosno "bezpokojęći", napriklad.

J:
Jediny vpis jest "zmiji", ale ako li ględěti na slova kde sut vyše jednoj sųglasky:
'bobŕji božji byčji člověčji delfińji děvičji gaďji gaďji golųbji govęďji gųśji jagnęťji jeleńji koźji kråvji kuŕji labęďji lěnji levji liśji medvěďji mråvji orľji pčeľji povsednji ptačji ptičji ranji sinji tvorčji vojevničji zmiji žučji'

Na moj russky poględ kråtka forma byla by "bobŕij božij byčij člověčij delfińij děvičij" i tako dalje, ale prividno južne jezyky takogo ne imajut.


In [109]:
" ".join(adjs[adjs['last'].str[-1] == "j"].isv.values)

'bobŕji božji byčji člověčji delfińji děvičji gaďji gaďji golųbji govęďji gųśji jagnęťji jeleńji koźji kråvji kuŕji labęďji lěnji levji liśji medvěďji mråvji orľji pčeľji povsednji ptačji ptičji ranji sinji tvorčji vojevničji zmiji žučji'

In [147]:
" ".join(
    adjs.query('last.str.len() >= 2 and last.str[-1] != "n" and last.str[-1] != "k" and last.str[-1] != "j"', engine='python').drop_duplicates(subset=['last']).isv.values
)

'azotisty běgly bezmozgy blizši bodry bogatši boljši bratov brzy bystry byvši črstvy črtov dědov divnějši dobry dȯlgy doråstly glųbši gȯrdy gorši hriply hytry izkonavči izpųkly každy kråljev kysly legši lěpši lučši medly menši mlådši mly mokry mŕtvy mųžev najmękši najvęćši najzly netrězvy nezly obći obly obmŕly opuhly pȯlny naděje prošly råbotči rad seksi světly tȯlsty tvŕdy vethy vysši zlověšći žȯlty'

In [148]:
adjs.query('last.str.len() >= 2 and last.str[-1] != "n" and last.str[-1] != "k" and last.str[-1] != "j"', engine='python').drop_duplicates(subset=['last'])

,id,isv,addition,en,ru,bg,hr,sr,frequency,pos,last
528,24409,azotisty,NaN,nitrous,азотистый,азотист,"dušikov, dušični",азотаст,0.299583,adj,st
681,20345,běgly,NaN,fluent,"беглый, свободный",свободно владеещ даден език,tečan,течан,1.345833,adj,gl
766,24646,bezmozgy,NaN,brainless,безмозглый,"безмозъчен, празноглав, тъп","praznoglav, šupljoglav, glup, ograničen","безмозган, глуп, блесав, празноглав",0.627917,adj,zg
948,6610,blizši,NaN,nearer,"ближе, более близкий",по-близо,bliži,ближи,2.0375,adj,zš
991,20355,bodry,NaN,"cheerful, brisk, spirited","бодрый, энергичный","весел, бодър, енергичен, жив","bodar, radostan, smio, veseo, živahan, energičan","бодар, радостан, смео, весео, живахан, енергичан",0.907917,adj,dr
997,6629,bogatši,NaN,richer,"богаче, более богатый",по-богат,bogatiji,богатији,1.639583,adj,tš
1054,35660,boljši,NaN,"bigger, greater, larger","больший, больше",по-голям,veći,већи,2.565417,adj,ljš
1111,24801,bratov,NaN,brother’s,"брата, братов, братний",братов,"bratov, bratovljev","братов, братовљев",0.950833,adj,bratov
1178,5583,brzy,NaN,"quick, fast","быстрый, скорый",бърз,brz,"брз, хитар",1.9175,adj,brz
1244,1762,bystry,NaN,"quick, fast",быстрый,бърз,brz,"брз, хитар",1.95125,adj,str


In [113]:
for word in adjs.query('last.str.len() == 2 and last.str[-1] != "n"', engine='python').drop_duplicates(subset=['last']).isv.values:
    print(word, "->", )

array(['azotisty', 'běgly', 'bezmozgy', 'blizky', 'blizši', 'bodry',
       'bogatši', 'božji', 'bridky', 'člověčji', 'delfińji', 'divnějši',
       'dobry', 'dȯlgy', 'gaďji', 'gȯrdy', 'goŕky', 'gorši', 'hriply',
       'izkonavči', 'jagnęťji', 'jarky', 'každy', 'klejky', 'koźji',
       'krasivši', 'kråtky', 'kråvji', 'krěhky', 'krěpky', 'kuŕji',
       'legky', 'legši', 'lěnji', 'lěpši', 'liśji', 'lučši', 'medly',
       'mękky', 'mělky', 'menši', 'mlådši', 'mly', 'mokry', 'najmękši',
       'najvęćši', 'nebesky', 'netrězvy', 'nezly', 'obći', 'obly',
       'opuhly', 'pčeľji', 'prošly', 'råbotči', 'seksi', 'sųvisly',
       'světly', 'tenky', 'tęžky', 'vethy', 'zlověšći', 'žȯlty'],
      dtype=object)

In [ ]:
for word in adjs.query('last.str.len() > 3 and last.str[-1] != "n"', engine='python').isv.values:
    print(word, "->", )

In [145]:
" ".join(adjs.query('last.str[-1] == "k" and last.str[-2] == "s"', engine='python').drop_duplicates(subset=['last']).isv.values)

'abhazsky afgansky akademičsky akrobatsky aktorsky alpijsky arabsky bahamsky bělorussky božsky braziľsky caŕsky češsky dekadentsky doľnosŕbsky egiptsky etiopsky filozofsky franksky freudovsky grådsky grenlandsky keltsky kiriličsky, kirilsky luksemburžsky nebesky neprijateljsky tadžičsky, tadžiksky velssky vųgrsky'

In [146]:
adjs[adjs.isv.str[-3:] == "eny"]

,id,isv,addition,en,ru,bg,hr,sr,frequency,pos,last
644,6483,basnjesloveny,NaN,terrific,"баснословный, потрясающий, великолепный","баснословен, потресаващ, великолепен",basnoslovan,баснослован,1.11875,adj,n
673,24576,bedreny,NaN,crural,бедренный,бедрен,bedreni,"бедрен, куков",0.52125,adj,n
1586,101,črveny,NaN,red,красный,червен,crven,црвен,2.1975,adj,n
1630,6939,čuvstveny,NaN,emotional,"чувственный, эмоциональный",емоционален,"čuvstven, emotivan, osjećajan","чувствен, осећајан",1.437917,adj,n
1992,7097,dobro obdarjeny,NaN,well-endowed,хорошо обеспеченный,добре обезпечен,dobro obdaren,добро обдарен,1.318333,adj,n
...,...,...,...,...,...,...,...,...,...,...,...
17452,2903,zasluženy,NaN,"worthy, well-deserved, meritorious","заслуженный, достойный",заслужен,zaslužan,заслужан,1.430417,adj,n
17465,23844,zasråmjeny,NaN,embarrassed,"застыженный, смущенный","засрамен, посрамен","zasramljen, posramljen","застиђен, осрамљен",0.5475,adj,n
17525,2894,zatvorjeny,NaN,"closed, shut","затворенный, закрытый, запертый",затворен,zatvoren,затворен,1.852083,adj,n
17602,752,zeleny,NaN,green,зеленый,зелен,zelen,зелен,2.300417,adj,n


In [99]:
" ".join(['aluminijevy', 'basnjevy', 'beževy', 'bojevy', 'cibuljevy',
       'deševy', 'dȯžďevy', 'dvulicevy', 'gruševy', 'koričnevy',
       'nastrojevy', 'niševy', 'oceljevy', 'oranževy', 'ośevy',
       'pljuševy', 'pomarančevy', 'roževy', 'sětevy', 'tydnjevy',
       'višnjevy', 'zamševy', 'drožđevy'])

'aluminijevy basnjevy beževy bojevy cibuljevy deševy dȯžďevy dvulicevy gruševy koričnevy nastrojevy niševy oceljevy oranževy ośevy pljuševy pomarančevy roževy sětevy tydnjevy višnjevy zamševy drožđevy'

In [85]:
adjs.query('last in @vc[@vc>=4].index', engine='python')

,id,isv,addition,en,ru,bg,hr,sr,frequency,pos,last
9,36565,abhazsky,NaN,"Abkhaz, Abkhazian",абхазский,абхазки,abhaski,абхазијски,0.444167,adj,zsk
26,6120,absolutny,NaN,"absolute, utter, sheer","абсолютный, полнейший","абсолютен, чист, пълен, истински","apsolutan, potpun, neograničen",апсолутан,1.17125,adj,tn
44,24071,adekvatny,NaN,adequate,"адекватный, удовлетворяющий",адекватен,"odgovarajući, prikladan, dovoljan, adekvatan","одговарајући, прикладан, довољан, адекватан",1.493333,adj,tn
48,17108,administrativny,NaN,administrative,административный,административен,administrativan,административан,1.669583,adj,vn
65,36348,aerodinamičny,NaN,"aerodynamic, streamlined",аэродинамический,аеродинамичен,aerodinamičan,аеродинамичан,0.974583,adj,čn
...,...,...,...,...,...,...,...,...,...,...,...
17934,-37025,gazovy,NaN,gas,газовый,!gasni,!gasni,"гасни, који користи гас",!,adj,v
17936,-37028,sněžny,NaN,snowy,"снежный, заснеженный",!снежно,"!snježna, snježna, snježna","снежни, снијежни, сњежни",!,adj,žn
17939,-37031,drožđevy,NaN,yeast,дрожжевой,!дрожди,!kvasci,"квашчев, који садржи квасац, од квасца",!,adj,v
17945,-37037,kvasny,NaN,yeast,дрожжевой,!дрожди,!kvasci,"квашчев, који садржи квасац, од квасца",!,adj,sn


In [56]:
slovnik[slovnik.partOfSpeech.str.contains("adj")].isv.str[-1].value_counts()

y    2732
i     139
n       5
v       5
a       2
d       2
e       1
Name: isv, dtype: int64

In [162]:
slovnik[(slovnik.partOfSpeech.str.contains("adj")) & (slovnik.isv.str[-1] != "y") & (slovnik.isv.str[-1] != "i") & (~slovnik.isv.str.contains(" "))][
    list(slovnik.columns[:6]) + ["ru", "sr", "bg"]
]

,id,isv,addition,partOfSpeech,type,en,ru,sr,bg
537,24425,babin,NaN,adj.,1,grandma’s,бабушкин,"бабин, бакин",бабин
1111,24801,bratov,NaN,adj.,1,brother’s,"брата, братов, братний","братов, братовљев",братов
1574,24984,črtov,NaN,adj.,1,devil’s,"чертов, дьявольский",ђавољи,дяволски
1722,25161,dědov,NaN,adj.,1,grandfather’s,"дедов, дедушкин, дедовский",дедин,дядов
2118,7179,dȯlžen,"(dȯlžna, dȯlžno)",adj.,3,"obliged, obligated, bound to, due",должен,"дужан, обавезан",длъжен
5448,26812,kråljev,NaN,adj.,1,king’s,"королевский, короля",краљев,кралски
6293,27423,mamin,NaN,adj.,1,mother’s,мамин,мамин,мамин
6371,27450,materin,NaN,adj.,1,mother’s,"мамин, матери, материн",мајчин,"мамин, майчин"
7001,27871,mųžev,NaN,adj.,1,"man’s, husband’s","мужской, мужнин","мужев, мужевљев","мъжки, съпружески"
11035,5999,povinen,"(povinny, povinna, povinno)",adj.,2,"ought to, should, obliged to, bound to, due","должен, обязан","дужан, обавезан","!трябва, трябва, длъжен, длъжен, по силата на"


In [172]:
slovnik[
    (slovnik.partOfSpeech.str.contains("adj")) & (slovnik.isv.str[-1] != "y") & (slovnik.isv.str[-1] == "i") & (slovnik.isv.str[-2:] != "ši") 
    & (slovnik.isv.str[-2] != "ć") & (slovnik.isv.str[-2] != "j") & (~slovnik.isv.str.contains(" "))][
    list(slovnik.columns[:6]) + ["ru", "sr", "bg"]
]

,id,isv,addition,partOfSpeech,type,en,ru,sr,bg
1344,610,ćuđi,NaN,adj.,1,#foreign,"чуждый, инородный, иностранный","стран, туђ, иностран, спољни",чуждестранен
1345,36629,ćuđi,NaN,adj.,nan,"another’s, someone else’s, alien",чужой,туђ,чужд
2430,4459,duži,NaN,adj.,2,great,"большой, крупный, великий","велик, велики, голем","голям, велик"
4042,5777,izkonavči,NaN,adj.,1,executive,исполнительный,извршан,изпълнителен
12355,20101,råbotči,NaN,adj.,1,"labour, labor","рабочий, трудовой",радни,"!на труда, на труда"
13309,30650,ryđi,NaN,adj.,1,red-haired,рыжий,"риђ, црвенокос",!червена
13506,32853,seksi,NaN,adj.,1,sexy,"сексуальный, сексуально привлекательный",секси,!секси
14791,5225,svěži,NaN,adj.,1,fresh,свежий,свеж,!пресни


In [169]:
slovnik[(slovnik.partOfSpeech.str.contains("adj")) & (slovnik.isv.str[-2:] == "ši")][
    list(slovnik.columns[:6]) + ["ru", "sr", "bg"]
]

,id,isv,addition,partOfSpeech,type,en,ru,sr,bg
948,6610,blizši,NaN,adj.,1,nearer,"ближе, более близкий",ближи,по-близо
997,6629,bogatši,NaN,adj.,1,richer,"богаче, более богатый",богатији,по-богат
1054,35660,boljši,NaN,adj.,2,"bigger, greater, larger","больший, больше",већи,по-голям
1260,2993,byvši,NaN,adj.,1,"former, erstwhile",бывший,бивши,бивш
1653,3032,daljši,NaN,adj.,1,"further, farther","дальнейший, последующий",даљи,"по-нататък, по-далеч"
1951,7063,divnějši,NaN,adj.,1,stranger,"более удивительный, более странный, удивительнее",чуднији,по-странен
3044,5867,glųbši,NaN,adj.,1,deeper,"глубже, более глубокий",дубљи,по-дълбоко
3156,1985,gorši,NaN,adj.,1,worse,"хуже, худший, более плохой",гори,по-лош
3443,7759,hlådnějši,NaN,adj.,1,colder,"холоднее, более холодный",хладнији,по-студено
5459,8660,krasivši,NaN,adj.,1,beautifuler,"красивее, более красивый",лепши,по-красив


In [42]:
slovnik[(slovnik.partOfSpeech.str.contains("adj")) & (slovnik.pos != "adj")][COLS]

,id,isv,addition,partOfSpeech,type,en,sameInLanguages,genesis,ru,be,...,hr,sl,cu,de,nl,eo,frequency,intelligibility,using_example,pos
12787,36454,råzpoznavajemy,NaN,#adj.,1,"identifiable, discernible, recognizable",v z j,NaN,"распознаваемый, идентифицируемый, различимый","распазнаваны, які распазнаецца, ідэнтыфікоўны,...",...,!prepoznatljiv,!prepoznavna,!,"!erkennbar, erkennbar, erkennbar","herkenbaar, identificeerbaar","!identigebla, videbla, rekonebla",1.21875,!be+ !bg+ !bs+ !cnr+ !cs+ !hr+ !mk+ !pl+ ru+ !...,NaN,None
